Here, we will use manually defined functions, as well as the pyAudioAnalysis library to featurize mp3 samples of audio data.

In [53]:
# import required modules 
import pandas as pd
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import audioFeatureExtraction
import sys,json, os 
import numpy as np

def stats(matrix):
    mean=np.mean(matrix)
    std=np.std(matrix)
    maxv=np.amax(matrix)
    minv=np.amin(matrix)
    median=np.median(matrix)

    output=np.array([mean,std,maxv,minv,median])
    
    return output

def convert_mono(filename):
    mono=filename[0:-4]+'_mono.wav'
    os.system('ffmpeg -i %s -ac 1 %s'%(filename,mono))
    return mono

def pyaudio_featurize(file):
    # use pyaudioanalysis library to export features
    # exported as file[0:-4].json 
    filename=file;
    mono=convert_mono(filename)
    [Fs, x] = audioBasicIO.readAudioFile(mono);
    F = audioFeatureExtraction.stFeatureExtraction(x, Fs, 0.050*Fs, 0.025*Fs);
    os.remove(mono)

    data={
        'features': F[0].tolist()
        }

    jsonfile=open(filename[0:-4]+'.json','w')
    print(jsonfile)
    json.dump(data,jsonfile)
    jsonfile.close() 
    
    # os.system('python pyaudio_help.py %s'%(file))
    jsonfile=file[0:-4]+'.json'
    g=json.load(open(jsonfile))
    features=np.array(g['features'])
    
    # now go through all the features and get statistical features for array
    new_features=list()
    all_labels=['zero crossing rate','energy','entropy of energy','spectral centroid',
                'spectral spread', 'spectral entropy', 'spectral flux', 'spectral rolloff',
                'mfcc1','mfcc2','mfcc3','mfcc4',
                'mfcc5','mfcc6','mfcc7','mfcc8',
                'mfcc9','mfcc10','mfcc11','mfcc12',
                'mfcc13','chroma1','chroma2','chroma3',
                'chroma4','chroma5','chroma6','chroma7',
                'chroma8','chroma9','chroma10','chroma11',
                'chroma12','chroma deviation']
    labels=list()
                
    for i in range(len(features)):
        tfeature=stats(features[i])
        for j in range(len(tfeature)):
            new_features.append(tfeature[j])
            if j==0:
                labels.append('mean '+all_labels[i])
            elif j==1:
                labels.append('std '+all_labels[i])
            elif j==2:
                labels.append('max '+all_labels[i])
            elif j==3:
                labels.append('min '+all_labels[i])
            elif j==4:
                labels.append('median '+all_labels[i])
            
    new_features=np.array(new_features)
    os.remove(jsonfile)
    
    return new_features, labels

In [54]:
url = "F:/dataset/dataset/train2.csv"
predictor_df = pd.read_csv(url)

We will take only the first 15 letters of the filename to get rid of the part of the path that defines the folder the files are in. We need to do this because of the way the loop is defined later.

In [55]:
predictor_df['path'] = predictor_df['path'].apply(lambda s: s[0:])

Not every sample is given an age, so we drop these samples. Moreover, we subset the dataframe to only include the filename and the age.

In [56]:
predictor_df = predictor_df[['path','age']]
predictor_df.dropna(inplace=True)
predictor_df

,path,age
0,common_voice_en_19664034.mp3,thirties
1,common_voice_en_19664035.mp3,thirties
2,common_voice_en_19664037.mp3,thirties
3,common_voice_en_19664038.mp3,thirties
4,common_voice_en_19664040.mp3,thirties
...,...,...
27987,common_voice_en_19411960.mp3,thirties
27988,common_voice_en_19411961.mp3,thirties
27989,common_voice_en_19411962.mp3,thirties
27990,common_voice_en_19411963.mp3,thirties


We create a loop to featurize each sample and we create a dataframe.

In [57]:
lst = []
extract_labels = pyaudio_featurize('F:/dataset/en/clips/common_voice_en_19664034.mp3')
labels = extract_labels[1]
for sample in list(predictor_df['path']):
    if(os.path.exists('F:/dataset/en/clips/%s'%(sample))):
        features = pyaudio_featurize('F:/dataset/en/clips/%s'%(sample))
        row = list(features[0])
        row.insert(0,sample)
        lst.append(row)
    else:
        break
    
    
labels.insert(0,'path')    
df = pd.DataFrame(lst,columns=labels)

<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19664034.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19664034.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19664035.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19664037.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19664038.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19664040.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_18433998.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_18433999.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_18434000.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_184

<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19156122.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19156123.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19156124.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19156125.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_18603711.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_18603712.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_18603713.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_18603714.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_18603715.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_199

<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_18422055.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_18422056.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_112550.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_112551.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_112552.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_112553.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_112554.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_20076095.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_20076096.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_20076097.json

<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19963202.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19963203.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19963204.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19963205.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19370625.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19370626.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19370627.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19370628.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19370629.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_186

<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19968073.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19968077.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19968078.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19968079.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19968080.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_18546968.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_18546969.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_18546973.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_18546977.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_178

...

<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19401202.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19401203.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19401204.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19401205.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19401206.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19401207.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19401208.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19401209.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19401219.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_194

<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19431184.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19431195.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19431196.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19431197.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19431198.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19431199.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19431215.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19431217.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19431219.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_194

<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411508.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411509.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411510.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411511.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411512.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411513.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411515.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411522.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411523.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_194

<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411767.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411769.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411771.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411772.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411773.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411784.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411785.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411786.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411787.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_194

<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411911.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411912.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411913.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411924.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411925.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411926.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411927.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411928.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_19411934.json' mode='w' encoding='cp1252'>
<_io.TextIOWrapper name='F:/dataset/en/clips/common_voice_en_194

In [58]:
df

,path,mean zero crossing rate,std zero crossing rate,max zero crossing rate,min zero crossing rate,median zero crossing rate,mean energy,std energy,max energy,min energy,...,mean chroma12,std chroma12,max chroma12,min chroma12,median chroma12,mean chroma deviation,std chroma deviation,max chroma deviation,min chroma deviation,median chroma deviation
0,common_voice_en_19664034.mp3,0.073258,0.074531,0.290538,0.000000,0.048353,0.020270,0.035007,0.269117,4.567455e-05,...,0.003188,0.006275,0.037782,1.287744e-07,0.000793,0.030306,0.024436,0.121409,0.000649,0.023972
1,common_voice_en_19664035.mp3,0.052858,0.051000,0.238433,0.000000,0.033556,0.024537,0.035153,0.158652,6.912406e-05,...,0.002471,0.004895,0.038482,3.533793e-06,0.000759,0.036536,0.025379,0.119332,0.002261,0.030440
2,common_voice_en_19664037.mp3,0.059639,0.058753,0.368070,0.003752,0.040850,0.020319,0.033536,0.237074,2.490513e-05,...,0.003548,0.006826,0.068758,3.455446e-05,0.001187,0.033004,0.025029,0.094953,0.001257,0.023595
3,common_voice_en_19664038.mp3,0.060613,0.053067,0.296373,0.001251,0.045436,0.023359,0.032630,0.147543,4.584024e-05,...,0.003207,0.005361,0.029206,2.182228e-05,0.001279,0.033231,0.026672,0.109625,0.000419,0.021328
4,common_voice_en_19664040.mp3,0.047078,0.050666,0.309296,0.000000,0.032930,0.026648,0.039978,0.177902,1.362596e-05,...,0.002990,0.005380,0.033914,1.442981e-06,0.001059,0.036865,0.025092,0.125037,0.000422,0.031240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27987,common_voice_en_19411960.mp3,0.076514,0.075043,0.243435,0.000000,0.035431,0.029220,0.038589,0.169370,1.038792e-10,...,0.006435,0.013743,0.105338,2.350297e-37,0.002238,0.023028,0.015873,0.100919,0.000539,0.021849
27988,common_voice_en_19411961.mp3,0.080600,0.073761,0.258024,0.013756,0.048770,0.035772,0.044525,0.203933,4.169727e-09,...,0.008653,0.015465,0.129077,3.552635e-05,0.002808,0.018399,0.013025,0.051424,0.000389,0.015187
27989,common_voice_en_19411962.mp3,0.087130,0.076696,0.250521,0.000000,0.049604,0.019898,0.028317,0.120220,8.576813e-10,...,0.004918,0.008724,0.074591,0.000000e+00,0.001690,0.018017,0.014445,0.064025,0.000643,0.015401
27990,common_voice_en_19411963.mp3,0.057272,0.061171,0.240517,0.000000,0.028762,0.039000,0.049269,0.201507,9.468018e-09,...,0.007743,0.014623,0.121270,1.160515e-37,0.002726,0.022794,0.013710,0.057029,0.000606,0.020183


We merge the two dataframes on the column 'filename' so we can create a DataFrame with the audio features, as well as the age category.

In [59]:
new_df = df.merge(predictor_df,on='path')

In [60]:
new_df

,path,mean zero crossing rate,std zero crossing rate,max zero crossing rate,min zero crossing rate,median zero crossing rate,mean energy,std energy,max energy,min energy,...,std chroma12,max chroma12,min chroma12,median chroma12,mean chroma deviation,std chroma deviation,max chroma deviation,min chroma deviation,median chroma deviation,age
0,common_voice_en_19664034.mp3,0.073258,0.074531,0.290538,0.000000,0.048353,0.020270,0.035007,0.269117,4.567455e-05,...,0.006275,0.037782,1.287744e-07,0.000793,0.030306,0.024436,0.121409,0.000649,0.023972,thirties
1,common_voice_en_19664035.mp3,0.052858,0.051000,0.238433,0.000000,0.033556,0.024537,0.035153,0.158652,6.912406e-05,...,0.004895,0.038482,3.533793e-06,0.000759,0.036536,0.025379,0.119332,0.002261,0.030440,thirties
2,common_voice_en_19664037.mp3,0.059639,0.058753,0.368070,0.003752,0.040850,0.020319,0.033536,0.237074,2.490513e-05,...,0.006826,0.068758,3.455446e-05,0.001187,0.033004,0.025029,0.094953,0.001257,0.023595,thirties
3,common_voice_en_19664038.mp3,0.060613,0.053067,0.296373,0.001251,0.045436,0.023359,0.032630,0.147543,4.584024e-05,...,0.005361,0.029206,2.182228e-05,0.001279,0.033231,0.026672,0.109625,0.000419,0.021328,thirties
4,common_voice_en_19664040.mp3,0.047078,0.050666,0.309296,0.000000,0.032930,0.026648,0.039978,0.177902,1.362596e-05,...,0.005380,0.033914,1.442981e-06,0.001059,0.036865,0.025092,0.125037,0.000422,0.031240,thirties
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27987,common_voice_en_19411960.mp3,0.076514,0.075043,0.243435,0.000000,0.035431,0.029220,0.038589,0.169370,1.038792e-10,...,0.013743,0.105338,2.350297e-37,0.002238,0.023028,0.015873,0.100919,0.000539,0.021849,thirties
27988,common_voice_en_19411961.mp3,0.080600,0.073761,0.258024,0.013756,0.048770,0.035772,0.044525,0.203933,4.169727e-09,...,0.015465,0.129077,3.552635e-05,0.002808,0.018399,0.013025,0.051424,0.000389,0.015187,thirties
27989,common_voice_en_19411962.mp3,0.087130,0.076696,0.250521,0.000000,0.049604,0.019898,0.028317,0.120220,8.576813e-10,...,0.008724,0.074591,0.000000e+00,0.001690,0.018017,0.014445,0.064025,0.000643,0.015401,thirties
27990,common_voice_en_19411963.mp3,0.057272,0.061171,0.240517,0.000000,0.028762,0.039000,0.049269,0.201507,9.468018e-09,...,0.014623,0.121270,1.160515e-37,0.002726,0.022794,0.013710,0.057029,0.000606,0.020183,thirties


In [61]:
new_df.to_csv('F:dataset/dataset/audio_training_data_cleaned3.csv') # export cleaned data as csv file